This notebook is designed to run in a IBM Watson Studio Apache Spark runtime. In case you are running it in an IBM Watson Studio standard runtime or outside Watson Studio, we install Apache Spark in local mode for test purposes only. Please don't use it in production.

In [ ]:
# !pip install --upgrade pip

In [ ]:
# if not ('sc' in locals() or 'sc' in globals()):
#     print('It seems you are note running in a IBM Watson Studio Apache Spark Notebook. You might be running in a IBM Watson Studio Default Runtime or outside IBM Waston Studio. Therefore installing local Apache Spark environment for you. Please do not use in Production')
    
#     from pip import main
#     main(['install', 'pyspark==2.4.5'])
    
#     from pyspark import SparkContext, SparkConf
#     from pyspark.sql import SparkSession

#     sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
    
#     spark = SparkSession \
#         .builder \
#         .getOrCreate()

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()

spark



In case you want to learn how ETL is done, please run the following notebook first and update the file name below accordingly

https://github.com/IBM/coursera/blob/master/coursera_ml/a2_w1_s3_ETL.ipynb


In [2]:
# delete files from previous runs
# !rm -f hmp.parquet*

# download the file containing the data in PARQUET format
# !wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
# create a dataframe out of it
df = spark.read.parquet('../w3/hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')

In [22]:
splits = df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [23]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler


indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")

normalizer = MinMaxScaler(inputCol="features", outputCol="features_norm")

In [30]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [31]:
from pyspark.ml import Pipeline

# pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, lr])
pipeline = Pipeline(stages=[indexer, vectorAssembler, lr])

In [32]:
model = pipeline.fit(df_train)

In [33]:
prediction = model.transform(df_train)

In [34]:
prediction.printSchema()

root
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- z: integer (nullable = true)
 |-- source: string (nullable = true)
 |-- class: string (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [35]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy")\
  .setPredictionCol("prediction").setLabelCol("label")
    
binEval.evaluate(prediction) 

0.2066235326067176

In [36]:
prediction = model.transform(df_test)

In [37]:
binEval.evaluate(prediction)  # 0.13003934176572612

0.20651833281103102